In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pprint import pprint

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

### Load Dataset

In [3]:
dataset = 'survey_report_combined.csv'
df = pd.read_csv(dataset, dtype=object)

In [4]:
df.head()

,year,original_missing_values_per_row_percentage,country,programming_languages,programming_languages_other,years_experience,occupation,age,os,job_satisfaction,company_size,want_work_language,gender,education,database_desire_work,database_worked_with,undergrad_major,remote_status,industry
0,2011,18.18,Africa,javascript,NaN,<2,Web Application Developer,< 20,Linux,FML,1-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Consulting
1,2011,27.27,Other Europe,java;sql;c,NaN,41310,Server Programmer,25-29,Windows 7,So happy it hurts,25-100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Software Products
2,2011,45.45,India,java;javascript;sql,NaN,41435,Server Programmer,25-29,Linux,NaN,100-999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Software Products
3,2011,9.09,Germany,java,haskell,41310,Student,< 20,Linux,I enjoy going to work,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Foundation / Non-Profit
4,2011,13.64,Other Asia,java;javascript;css;php;sql;c++;c;perl,NaN,11,"Executive (VP of Eng., CTO, CIO, etc.)",35-39,Linux,It pays the bills,1-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Software Products


### Exploding programming languages feature
Two columns: `programming_languages` and `programming_languages_other`

In [5]:
df['programming_languages']       = df.programming_languages.str.lower().str.split('/|;')
df['programming_languages_other'] = df.programming_languages_other.str.lower().str.split('/|;')

In [6]:
df_explode = df.explode('programming_languages')
df_explode['programming_languages'] = df_explode.programming_languages.str.strip()

In [7]:
df_explode = df_explode.explode('programming_languages_other')
df_explode['programming_languages_other'] = df_explode.programming_languages_other.str.strip()

In [8]:
df_explode.head()

,year,original_missing_values_per_row_percentage,country,programming_languages,programming_languages_other,years_experience,occupation,age,os,job_satisfaction,company_size,want_work_language,gender,education,database_desire_work,database_worked_with,undergrad_major,remote_status,industry
0,2011,18.18,Africa,javascript,NaN,<2,Web Application Developer,< 20,Linux,FML,1-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Consulting
1,2011,27.27,Other Europe,java,NaN,41310,Server Programmer,25-29,Windows 7,So happy it hurts,25-100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Software Products
1,2011,27.27,Other Europe,sql,NaN,41310,Server Programmer,25-29,Windows 7,So happy it hurts,25-100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Software Products
1,2011,27.27,Other Europe,c,NaN,41310,Server Programmer,25-29,Windows 7,So happy it hurts,25-100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Software Products
2,2011,45.45,India,java,NaN,41435,Server Programmer,25-29,Linux,NaN,100-999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Software Products


In [9]:
df_explode.shape

(2060544, 19)

### JS related technology

In [10]:
other_lang_js = df_explode[df_explode.programming_languages_other.str.endswith('js', na=False)]
other_lang_js.programming_languages_other.value_counts().to_frame()

,programming_languages_other
ember js,199
react js,182
knockout js,170
backbone js,148
angular js,107
meteor js,104
ext js,89
node js,47
d3 js,22
underscore js,18


In [11]:
df_explode[df_explode.programming_languages.str.endswith('js', na=False)].programming_languages.value_counts().to_frame()

,programming_languages
node js,12884
angular js,11824
react js,2541


### Filtering common programming languages

In [12]:
common_programming_languages = df_explode.programming_languages.str.lower().unique().tolist()